In [1]:
import pandas as pd
from Transformer.Config import Config
from Transformer.Model import Transformer
import sentencepiece as spm
import tensorflow_datasets as tfds
import torch.nn as nn
from tqdm import tqdm, tqdm_notebook, trange

df=pd.read_csv("ex1.csv")

questions=[i for i in df["q"]]
answers=[i for i in df["a"]]

ModuleNotFoundError: No module named 'Transformermy'

In [3]:

import requests

# ﻿﻿《밑바닥부터 시작하는 딥러닝2》역자 깃허브에서 데이터를 가져옵니다.
url = "https://raw.githubusercontent.com/WegraLee/deep-learning-from-scratch-2/master/dataset/date.txt"
r = requests.get(url)

questions, answers = [], []
for line in r.text.strip().split('\n'):
    idx = line.find('_')
    questions.append(line[:idx].strip())
    answers.append(line[idx:].strip())

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
class TokenizerPlus(Tokenizer):
        
    def to_tensor(self, texts, **kwargs):
        sequences = self.texts_to_sequences(texts)
        padded = pad_sequences(sequences, **kwargs)
        return torch.tensor(padded, dtype=torch.int64)
    
    def to_string(self, tensor):
        texts = self.sequences_to_texts(tensor.data.numpy())
        return [t[::2] for t in texts]
    

# 토큰으로 나눠주고 패딩 처리를 합니다.
tokenizer = TokenizerPlus(char_level=True, filters='')
tokenizer.fit_on_texts(questions)
tokenizer.fit_on_texts(answers)

src = tokenizer.to_tensor(questions)
tgt = tokenizer.to_tensor(answers)

In [5]:
print(src.shape,tgt.shape)

torch.Size([50000, 29]) torch.Size([50000, 11])


In [6]:
def greedy_decoding(trm, src, start_token):
    N = src.size(0)
    preds = tokenizer.to_tensor([start_token]*N)

    for _ in range(10):
        y_pred,a,b,c = trm(src,preds)
        t_pred = torch.argmax(y_pred[:,-1,:], axis=-1, keepdims=True)
        preds = torch.cat([preds, t_pred], axis=1)            
    
    return tokenizer.to_string(preds[:, 1:])

In [7]:
from torch import optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
class Trainer:
    def __init__(self, model, loss_fn, optimizer, pad_id=0, start_token='_'):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.pad_id = pad_id
        self.start_token = start_token
        self.losses = []
        self.current_epoch = 0
        
    def train(self, src, tgt, max_epoch=1, batch_size=64):
        X1_train = src
        X2_train = tgt[:, :-1]
        y_train = tgt[:, 1:]
        ds = TensorDataset(X1_train, X2_train, y_train)
        loader = DataLoader(ds, batch_size=batch_size, shuffle=True)
        
        for epoch in range(1, max_epoch+1):
            self.model.train()
            running_loss = 0
            self.current_epoch += 1
            print("EPOCH: %s :: " %self.current_epoch, end='')
            for i, (x1, x2, yy) in enumerate(loader):
                if(i%8==0):
                    print(i,end="/")
                y_pred,ea,de,eda = self.model(x1, x2)
                loss = self.loss_fn(y_pred.view(-1, y_pred.size(-1)), yy.view(-1))
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                running_loss += loss.item()
            
            self.model.eval()
            print("eval")
            running_loss = round(running_loss / (i+1), 3)
            print("running loss")
            self.losses.append(running_loss)
            print("losses")
            acc = self.evaluate(src[:20], y_train[:20])
            print("Loss: %s" %(running_loss), "ACC: %s" %acc)

    def evaluate(self, src, y):
        pred = np.array(greedy_decoding(self.model,src, start_token='_'))
        y_text = np.array(tokenizer.to_string(y))
        acc = (pred == y_text).sum() / y_text.size
        return acc

In [9]:

from Transformer.Config import Config
config=Config(len(tokenizer.word_index)+1)
config.n_enc_seq=64
config.n_dec_seq=64
config.d_hidn=64
config.d_ff=128
config.d_head=64

config.n_layer=2
print(config)
print(answers[0])

ModuleNotFoundError: No module named 'Transformermy'

In [5]:
print(type(src),(type(tgt)))
print(src.shape,tgt.shape)

<class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([50000, 29]) torch.Size([50000, 11])


In [58]:
from Transformer.Model import Transformer
config=Config(len(tokenizer.word_index)+1)

model = Transformer(config)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

trainer = Trainer(model, loss_fn, optimizer)
trainer.train(src, tgt, max_epoch=1)

EPOCH: 1 :: 0/8/16/24/32/40/48/56/64/72/80/88/96/104/112/120/128/136/144/152/160/168/176/184/192/200/208/216/224/232/240/248/256/264/272/280/288/296/304/312/320/328/336/344/352/360/368/376/384/392/400/408/416/424/432/440/448/456/464/472/480/488/496/504/512/520/528/536/544/552/560/568/576/584/592/600/608/616/624/632/640/648/656/664/672/680/688/696/704/712/720/728/736/744/752/760/768/776/eval
running loss
losses
Loss: 0.116 ACC: 0.95


In [83]:
for i in src[:2]:
    print(str(tokenizer.to_string(i.view(1,29))),end=" / ")
    print( greedy_decoding(model,i.view(1,29),"_"))

['september 27, 1994'] / ['1994-09-27']
['august 19, 2003'] / ['2003-08-19']


In [86]:

input=tokenizer.to_tensor("august 10, 1994").view(1,-1)
print(input)
print(input.shape,src.shape)
input = torch.cat([torch.zeros(1, src.size(1) - input.size(1)) , input], dim=1).to(torch.int64)

print(input,"\n",input.shape,src.shape)
print(str(tokenizer.to_string(input)))
output=greedy_decoding(model,input,"_")
print(output)


tensor([[11, 19, 36, 19, 24, 25,  6,  1,  2,  8,  6,  1,  4,  4, 16]])
torch.Size([1, 15]) torch.Size([50000, 29])
tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 11, 19, 36, 19,
         24, 25,  6,  1,  2,  8,  6,  1,  4,  4, 16]]) 
 torch.Size([1, 29]) torch.Size([50000, 29])
['august 10, 1994']
['1994-08-10']


In [31]:
def greedy_decoding(trm, src, start_token):
    N = src.size(0)
    preds = tokenizer.to_tensor([start_token]*N)

    for _ in range(10):
        y_pred,a,b,c = trm(src,preds)
        t_pred = torch.argmax(y_pred[:,-1,:], axis=-1, keepdims=True)
        preds = torch.cat([preds, t_pred], axis=1)            
    
    return tokenizer.to_string(preds[:, 1:])